In [46]:
import pymysql

from wmfdata import hive, mariadb

import pandas as pd
import datetime as dt

In [163]:
## Configuration

wikis = ['enwiki', 'frwiki', 'dewiki', 'eswiki', 'ruwiki',
         'itwiki', 'nlwiki', 'jawiki', 'zhwiki', 'ptwiki']

start_time = dt.datetime(2017, 1, 1, 0, 0, 0)
end_time = dt.datetime(2019, 3, 1, 0, 0, 0)

## Start time for the monthly count of users who add their first name to the Echo blacklist
## is moved to 2018-01-01 to remove a spike in usage in late 2017, as we are more interested
## more recent development as well as changes over time (which the spike hides). See also notes below.
## This is set as a string so it can be used in R.
echo_start_time = "2018-01-01"

## Name of the log databases for preference modifications:
log_tables = ['PrefUpdate_5563398_15423246', 'PrefUpdate_5563398']

## Format strings:
## MediaWiki database timestamp format
mw_format = "%Y%m%d%H%M%S"

## Standard format on timestamps in the Data Lake
hive_format = "%Y-%m-%dT%H:%M:%S"

## Graphs go into the "graphs/" folder
graph_folder = "graphs/"

## Graph suffix (always ".png" for now)
graph_suffix = ".png"

## Prefixes for all graphs, will have "_", the wiki db name, and the suffix appended to it.
## Titles for all graphs will have the wiki db name appended to it
disablemail_graph = "disablemail_monthly"
disablemail_title = "Monthly count of users disabling email - "

newuser_email_graph = "disable_newuser_email_monthly"
newuser_title = "Monthly count of users disabling email from brand-new users - "

email_blocklist_graph = "email_blocklist_initial_users_monthly"
email_blocklist_title = "Monthly count of users adding first name to email blacklist - "

echo_blocklist_graph = "notification_blocklist_initial_users_monthly"
echo_blocklist_title = "Monthly count of users adding first name to notification blacklist - "

email_usage_graph = "email_blocklist_histogram"
email_usage_title = "Number of users on email blacklist - "

echo_usage_graph = "echo_blocklist_histogram"
echo_usage_title = "Number of users on Echo notifications blacklist - "

In [4]:
%load_ext rpy2.ipython

In [5]:
%%R
library(data.table);
library(ggplot2);
library(tidyr);

/home/nettrom/venv/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: data.table 1.11.8  Latest news: r-datatable.com

  warnings.warn(x, RRuntimeWarning)


In [53]:
# The second function needs dnspython to work
import dns.resolver
import glob

def get_mediawiki_section_dbname_mapping(mw_config_path, use_x1):
    db_mapping = {}
    if use_x1:
        dblist_section_paths = [mw_config_path.rstrip('/') + '/dblists/all.dblist']
    else:
        dblist_section_paths = glob.glob(mw_config_path.rstrip('/') + '/dblists/s[0-9]*.dblist')
    for dblist_section_path in dblist_section_paths:
        with open(dblist_section_path, 'r') as f:
            for db in f.readlines():
                db_mapping[db.strip()] = dblist_section_path.strip().rstrip('.dblist').split('/')[-1]

    return db_mapping


def get_dbstore_host_port(db_mapping, use_x1, dbname):
    if dbname == 'staging':
        shard = 'staging'
    elif use_x1:
        shard = 'x1'
    else:
        try:
            shard = db_mapping[dbname]
        except KeyError:
            raise RuntimeError("The database {} is not listed among the dblist files of the supported sections."
                               .format(dbname))
    answers = dns.resolver.query('_' + shard + '-analytics._tcp.eqiad.wmnet', 'SRV')
    host, port = str(answers[0].target), answers[0].port
    return (host,port)

wikidb_map = get_mediawiki_section_dbname_mapping('/srv/mediawiki-config', False)

## Users turning off email

I'll interpret this statistics so as to mean we count whenever a user changes the "disablemail" preference to "true". Thus, if a user at some point decides to turn it off and on again, we'll count them again.

In [6]:
disablemail_query = '''
SELECT log_date, wiki, SUM(num_events) AS num_events
FROM ((
SELECT DATE_FORMAT(timestamp, '%Y-%m-01') AS log_date,
       wiki,
       COUNT(*) AS num_events
FROM log.{tables[0]}
WHERE timestamp >= "{start_timestamp}"
AND timestamp < "{end_timestamp}"
AND wiki IN ({wiki_list})
AND event_property = "{prop}"
AND event_value = "{value}"
GROUP BY log_date, wiki
)
UNION ALL
(
SELECT DATE_FORMAT(timestamp, '%Y-%m-01') AS log_date,
       wiki,
       COUNT(*) AS num_events
FROM log.{tables[1]}
WHERE timestamp >= "{start_timestamp}"
AND timestamp < "{end_timestamp}"
AND wiki IN ({wiki_list})
AND event_property = "{prop}"
AND event_value = "{value}"
GROUP BY log_date, wiki
)) AS e
GROUP BY log_date, wiki
'''

In [149]:
disablemail_counts = mariadb.run(disablemail_query.format(
    prop = "disablemail",
    value = "true",
    tables = log_tables,
    start_timestamp = start_time.strftime(mw_format),
    end_timestamp = end_time.strftime(mw_format),
    wiki_list = ','.join(['"{}"'.format(w) for w in wikis])
    ), host='logs'
)

In [150]:
disablemail_counts.tail(10)

,log_date,wiki,num_events
250,2019-02-01,dewiki,88.0
251,2019-02-01,enwiki,545.0
252,2019-02-01,eswiki,44.0
253,2019-02-01,frwiki,25.0
254,2019-02-01,itwiki,32.0
255,2019-02-01,jawiki,45.0
256,2019-02-01,nlwiki,14.0
257,2019-02-01,ptwiki,14.0
258,2019-02-01,ruwiki,66.0
259,2019-02-01,zhwiki,52.0


In [151]:
%%R

make_graphs = function(dt, wikis, title, filename, graph_dir, graph_suffix) {
    graphs = c()
    for(w in wikis) {
        g = ggplot(dt[wiki == w], aes(x = log_date, y = num_events)) +
        scale_x_date(date_breaks = "3 months", date_minor_breaks = "1 month",
                     date_labels = "%Y-%m") +
        expand_limits(y = 0) +
        labs(title = paste0(title, w),
             x = 'Date',
             y = 'Count') +
        theme_light(base_size = 14) +
        geom_line();
        ggsave(paste0(graph_dir, filename, '_', w, graph_suffix),
               plot = g, width = 30, height = 20, units = "cm", dpi = "screen");
        graphs = append(graphs, g);
    }
    graphs;
}

In [152]:
%%R -i disablemail_counts,wikis,disablemail_title,disablemail_graph,graph_folder,graph_suffix

disablemail_counts = data.table(disablemail_counts);
disablemail_counts[, log_date := as.Date(log_date)];

graphs = make_graphs(disablemail_counts, wikis, disablemail_title, disablemail_graph,
            graph_folder, graph_suffix);

## Users turning off email from brand new users

Similarly as for users disabling email (in general), I'll interpret this to mean the count of number of users who change this preference, again meaning that someone who turns it off and on again will be counted again.

In [153]:
disablenew_counts = mariadb.run(disablemail_query.format(
    prop = "email-allow-new-users",
    value = "false",
    tables = log_tables,
    start_timestamp = start_time.strftime(mw_format),
    end_timestamp = end_time.strftime(mw_format),
    wiki_list = ','.join(['"{}"'.format(w) for w in wikis])
    ), host='logs'
)

In [154]:
%%R -i disablenew_counts,wikis,newuser_title,newuser_email_graph,graph_folder,graph_suffix

disablenew_counts = data.table(disablenew_counts);
disablenew_counts[, log_date := as.Date(log_date)];

dates = seq.Date(min(disablenew_counts$log_date), max(disablenew_counts$log_date), by='month');
dates = data.table(log_date = dates);

# disablenew_counts = disablenew_counts[dates, on = 'log_date'];

graphs = make_graphs(disablenew_counts, wikis, newuser_title, newuser_email_graph,
            graph_folder, graph_suffix);

## Users who add their first user to the email block list

I'll interpret this to mean the first recorded timestamp of a given user setting their email block list to something that isn't empty. This leads to an expectancy problem, in that if we only look at events from `start_time`, we'll likely pick up experienced users just updating the list (rather than starting it). There's several ways we can remedy that problem, here are a couple:

1. Start our data gathering earlier than `start_time`, and filter out users who updated the list.
2. Migrate to the Data Lake and use the `mediawiki_user_history` table to identify when users registered and only count certain groups of users (e.g. users have to initalize the list within some time after registration).

In this case, we'll start with the first approach and gather data for a year prior to `start_time`, and remove those from the calculation. A year should be a decent amount of time to allow someone to start using the list.


In [155]:
email_blacklist_query = '''
SELECT wiki,
       DATE_FORMAT(first_event, "%Y-%m-01") AS log_date,
       COUNT(*) AS num_events
FROM (SELECT wiki, user_id, MIN(timestamp) AS first_event
      FROM ((
        SELECT wiki,
               event_userid AS user_id,
               MIN(timestamp COLLATE utf8_unicode_ci) AS timestamp
        FROM log.{tables[0]}
        WHERE timestamp >= "{data_timestamp}"
        AND timestamp < "{end_timestamp}"
        AND wiki IN ({wiki_list})
        AND event_property = "email-blacklist"
        AND event_value != "null"
        AND event_value != '"0"'
        AND event_value != '""'
        GROUP BY wiki, user_id
    )
    UNION ALL
    (
        SELECT wiki,
               event_userid AS user_id,
               MIN(timestamp) AS timestamp
        FROM log.{tables[1]}
        WHERE timestamp >= "{data_timestamp}"
        AND timestamp < "{end_timestamp}"
        AND wiki IN ({wiki_list})
        AND event_property = "email-blacklist"
        AND event_value != "null"
        AND event_value != '"0"'
        AND event_value != '""'
        GROUP BY wiki, user_id
    )) AS e
    GROUP BY wiki, user_id) AS ev
WHERE first_event >= "{start_timestamp}"
AND first_event < "{end_timestamp}"
GROUP BY wiki, log_date
'''

So… digging into this, I've found lots of duplicate events in the database. For example, one user has 4,588 events logged in the first three months of 2018, and in the first 100 of them there are lots of pairs of events with the same timestamp in the database but different save timestamps, and where the value of the blacklist doesn't change.

This means that my approach of using the first timestamp might just be working. It also means that there is no straightforward way of measuring usage of this feature (i.e. count of users modifying it) as it'll require deduplication.

I've also looked into whether this is an issue for the `disablemail` (turning it on) and `email-allow-new-users` settings (turning it off) for the same three-month period, and did there not find any discrepancies. The user with the most number of events had 3, the rest of the top 25 had 2, which seems perfectly reasonable over that time period.

In [156]:
email_blacklists = mariadb.run(email_blacklist_query.format(
    tables = log_tables,
    data_timestamp = start_time.replace(year=start_time.year-1).strftime(mw_format),
    start_timestamp = start_time.strftime(mw_format),
    end_timestamp = end_time.strftime(mw_format),
    wiki_list = ','.join(['"{}"'.format(w) for w in wikis])
    ), host='logs'
)

In [157]:
%%R -i email_blacklists,wikis,email_blocklist_title,email_blocklist_graph,graph_folder,graph_suffix

## 

email_blacklists = data.table(email_blacklists);
email_blacklists[, log_date := as.Date(log_date)];

dates = seq.Date(min(email_blacklists$log_date), max(email_blacklists$log_date), by='month');
dates = rbindlist(lapply(wikis, function(w) { data.table(wiki=rep(w, length(dates)), log_date=dates)}))

email_blacklists = merge(dates, email_blacklists, on = c('wiki', 'log_date'), all.x = TRUE);
email_blacklists[is.na(num_events), num_events := 0];

graphs = make_graphs(email_blacklists, wikis, email_blocklist_title, email_blocklist_graph,
                     graph_folder, graph_suffix);


Looking at the graphs, I was concerned about the spike in usage for July 2018. We discussed in the team whether it was initial usage or not, but after digging further and adding some additional clauses to the SQL query (e.g. the check for ID 0 and an empty quoted string), the cleaner data did not contain this spike.

## Users who add their first user to the Echo notification block list

Same approach as for the email blacklist, except the property is different.

In [43]:
echo_blacklist_query = '''
SELECT wiki,
       DATE_FORMAT(first_event, "%Y-%m-01") AS log_date,
       COUNT(*) AS num_events
FROM (SELECT wiki, user_id, MIN(timestamp) AS first_event
      FROM ((
        SELECT wiki,
               event_userid AS user_id,
               MIN(timestamp COLLATE utf8_unicode_ci) AS timestamp
        FROM log.{tables[0]}
        WHERE timestamp >= "{data_timestamp}"
        AND timestamp < "{end_timestamp}"
        AND wiki IN ({wiki_list})
        AND event_property = "echo-notifications-blacklist"
        AND event_value != "null"
        AND event_value != '"0"'
        AND event_value != '""'
        GROUP BY wiki, user_id
    )
    UNION ALL
    (
        SELECT wiki,
               event_userid AS user_id,
               MIN(timestamp) AS timestamp
        FROM log.{tables[1]}
        WHERE timestamp >= "{data_timestamp}"
        AND timestamp < "{end_timestamp}"
        AND wiki IN ({wiki_list})
        AND event_property = "echo-notifications-blacklist"
        AND event_value != "null"
        AND event_value != '"0"'
        AND event_value != '""'
        GROUP BY wiki, user_id
    )) AS e
    GROUP BY wiki, user_id) AS ev
WHERE first_event >= "{start_timestamp}"
AND first_event < "{end_timestamp}"
GROUP BY wiki, log_date
'''

In [158]:
echo_blacklists = mariadb.run(echo_blacklist_query.format(
    tables = log_tables,
    data_timestamp = start_time.replace(year=start_time.year-1).strftime(mw_format),
    start_timestamp = start_time.strftime(mw_format),
    end_timestamp = end_time.strftime(mw_format),
    wiki_list = ','.join(['"{}"'.format(w) for w in wikis])
    ), host='logs'
)

In [164]:
%%R -i echo_blacklists,wikis,echo_blocklist_title,echo_blocklist_graph,graph_folder,graph_suffix,echo_start_time

## 

echo_blacklists = data.table(echo_blacklists);
echo_blacklists[, log_date := as.Date(log_date)];

dates = seq.Date(min(echo_blacklists$log_date), max(echo_blacklists$log_date), by='month');
dates = rbindlist(lapply(wikis, function(w) { data.table(wiki=rep(w, length(dates)), log_date=dates)}))

echo_blacklists = merge(dates, echo_blacklists, on = c('wiki', 'log_date'), all.x = TRUE);
echo_blacklists[is.na(num_events), num_events := 0];

graphs = make_graphs(
    echo_blacklists[log_date >= echo_start_time],
    wikis, echo_blocklist_title, echo_blocklist_graph, graph_folder, graph_suffix);


In this case, due to how I am looking for the first event where the list contained something, I suspect that this data quality issue does not affect this data. The spikes that show up in the graphs can be explained as the feature being launched and lots of users making their first changes.

That being said, we're interested in the development of the graph over time, something that spike hides because the magnitude of later changes is diminished. We are also perhaps more interested in recent development relative to initial adoption. As a result, I choose to change the start date of these graphs from the beginning of the data we have to 2018-01-01, which removes the spike.

# Snapshot statistics

We are interested in understanding to what extent users have the same usernames on both the email and Echo notification block lists. Secondly, we want to know more about how many usernames are on the list, either as a graph of the distribution or some summary statistics. Let's tackle the list similarity measurement first.

For these measurements, we use whatever data is available in the replicated MediaWiki databases when the queries were run. Currently that's 2019-03-11.

## List similarity

I first ran a couple of queries to understand what values exists in the database, in this case I used the English Wikipedia as my test wiki:

```mysql
SELECT SUM(IF(up_value IS NULL, 1, 0)) AS num_is_null,
SUM(IF(up_value = "", 1, 0)) AS num_empty_string,
SUM(IF(up_value IS NOT NULL AND up_value != '', 1, 0)) AS num_set
FROM user_properties
WHERE up_property= 'echo-notifications-blacklist';
       
SELECT SUM(IF(up_value IS NULL, 1, 0)) AS num_is_null,
SUM(IF(up_value = "", 1, 0)) AS num_empty_string,
SUM(IF(up_value IS NOT NULL AND up_value != '', 1, 0)) AS num_set
FROM user_properties
WHERE up_property= 'email-blacklist';
```

Here, I found that no user has it set to `NULL`. A lot of users has it set to an empty string, whereas a lot fewer users has it set. For email, 58,016 users has it set to an empty string, while only 196 users has it set to something else. For Echo notifications, 5,520 users has it set to an empty string, while 956 users has it set to something else.

This suggests that if we simply did straight comparisons, we might get a lot of false positives in that a user will have both set to an empty string. In this case, based on the phrasing of the question, I'll interpret having an empty string as out of scope. In other words, we're measuring the size of the intersection between both lists in the case that either list is empty.

Further inspection of how `up_value` is set, it appears that these are unsorted lists, meaning that we need to split and sort them to properly compare their equality, as it's not clear that the list will be identical in both cases. (**NOTE:** Further inspection of the data suggests that this assumption might be wrong and that direct comparison is possible, it just wasn't obvious that the list is sorted. However, on English Wikipedia the set comparison method return 15 users, while direct comparison only returns 13.) That means we'll grab the data for all users and process it individually outside of MariaDB, as its support for splitting strings and such appears to be non-existent.

In [160]:
class PropUser:
    def __init__(self, user_id, email_set, echo_set):
        self.user_id = user_id
        self.email_set = email_set
        self.echo_set = echo_set
        
    def is_equal(self):
        '''
        The sets are equal if the intersection is the same length as the union.
        '''
        ## If either of them are not set, we return false
        if(len(self.email_set) == 0 or len(self.echo_set) == 0):
            return(False)

        return(len(self.email_set & self.echo_set) == len(self.email_set | self.echo_set))
    
    def one_set(self):
        '''
        Does the user have at least one user on either of their lists?        
        '''
        
        if(len(self.email_set) > 0):
            return(True)
        
        if(len(self.echo_set) > 0):
            return(True)
        
        return(False)

def calc_intersection(wiki, db_mapping,
                      email_prop = 'email-blacklist', echo_prop = 'echo-notifications-blacklist'):
    '''
    Calculate the proportion of users in the given wiki who have the same set of usernames in
    their email block list (`email_prop`) and Echo notifications block list (`echo_prop`).
    '''
    
    ## Query to retrieve user id and preference value for a given property where
    ## the value cannot be NULL nor an empty string
    prop_query = '''
    SELECT up_user, up_value
    FROM user_properties
    WHERE up_property = %s
    AND up_value IS NOT NULL
    AND up_value != ""
    '''
    
    ## Mapping of user ID to the PropUser object
    id_user_map = {}
           
    # connect to the database
    wiki_hostport = get_dbstore_host_port(db_mapping, False, wiki)
    wiki_dbconn = pymysql.connect(
        host = wiki_hostport[0],
        port = wiki_hostport[1],
        db= wiki,
        read_default_file = '/etc/mysql/conf.d/research-client.cnf',
        charset = 'utf8'
    )
    
    # grab all users who have email_prop set and process them
    with wiki_dbconn.cursor() as db_cursor:
        db_cursor.execute(prop_query,
                          (email_prop))
        for (user_id, prop_value) in db_cursor:
            prop_value = prop_value.decode('utf-8')
            props = set(prop_value.strip().split('\n'))
            id_user_map[user_id] = PropUser(user_id, props, set())
    
    # grab all users who have echo_prop set and process them
    with wiki_dbconn.cursor() as db_cursor:
        db_cursor.execute(prop_query,
                          (echo_prop))
        for (user_id, prop_value) in db_cursor:
            prop_value = prop_value.decode('utf-8')
            props = set(prop_value.strip().split('\n'))
            
            if(user_id in id_user_map):
                user = id_user_map[user_id]
                user.echo_set = props
            else:
                id_user_map[user_id] = PropUser(user_id, set(), props)

    num_users = len([u for u in id_user_map.values() if u.one_set()])
    num_equal = len([u for u in id_user_map.values() if u.is_equal()])
    
    print("""
    Wiki: {}
    Number of users with at least one value set: {}
    Number of users with both sets equal: {}
    Percentage: {}
    """.format(wiki, num_users, num_equal, 100*num_equal/num_users))
                
    wiki_dbconn.close()
    return(id_user_map)
    

In [161]:
for wiki in wikis:
    calc_intersection(wiki, wikidb_map)


    Wiki: enwiki
    Number of users with at least one value set: 1117
    Number of users with both sets equal: 15
    Percentage: 1.3428827215756491
    

    Wiki: frwiki
    Number of users with at least one value set: 164
    Number of users with both sets equal: 1
    Percentage: 0.6097560975609756
    

    Wiki: dewiki
    Number of users with at least one value set: 500
    Number of users with both sets equal: 3
    Percentage: 0.6
    

    Wiki: eswiki
    Number of users with at least one value set: 209
    Number of users with both sets equal: 2
    Percentage: 0.9569377990430622
    

    Wiki: ruwiki
    Number of users with at least one value set: 142
    Number of users with both sets equal: 4
    Percentage: 2.816901408450704
    

    Wiki: itwiki
    Number of users with at least one value set: 71
    Number of users with both sets equal: 0
    Percentage: 0.0
    

    Wiki: nlwiki
    Number of users with at least one value set: 54
    Number of users with both 

## List distribution

Here, we'll get the list for each user, decode it and turn it into a number. This can all be done in Pandas, I think.

In [118]:
def len_prop_list(x):
    if(x == ""):
        return(0)    
    return(len(x.strip().split('\n')))

def get_prop_dist(wiki, db_mapping, prop_name):
    '''
    For the given wiki, connect to the replicated database and grab all users with the given property
    set. Then return a `pandas.DataFrame` with user IDs and the length of their list.
    '''
    
    prop_query = '''
    SELECT up_user, up_value
    FROM user_properties
    WHERE up_property = "{}"
    AND up_value IS NOT NULL
    '''
    
    # connect to the database
    wiki_hostport = get_dbstore_host_port(db_mapping, False, wiki)
    wiki_dbconn = pymysql.connect(
        host = wiki_hostport[0],
        port = wiki_hostport[1],
        db= wiki,
        read_default_file = '/etc/mysql/conf.d/research-client.cnf',
        charset = 'utf8'
    )
    
    user_data = pd.read_sql_query(prop_query.format(prop_name), wiki_dbconn)
    user_data = user_data.applymap(try_decode).rename(columns = try_decode)
    user_data['n_users'] = user_data['up_value'].apply(len_prop_list)
    user_data = user_data.drop('up_value', axis = 1)
    user_data['wiki'] = wiki
    
    wiki_dbconn.close()
    
    return(user_data)
    

In [122]:
email_usage = pd.concat([get_prop_dist(w, wikidb_map, 'email-blacklist') for w in wikis])

In [123]:
echo_usage = pd.concat([get_prop_dist(w, wikidb_map, 'echo-notifications-blacklist') for w in wikis])

In [143]:
%%R

make_histograms = function(dt, wikis, title, filename, graph_dir, graph_suffix) {
    graphs = c()
    for(w in wikis) {
        g = ggplot(dt[wiki == w & n_users > 0], aes(x = n_users)) +
        labs(title = paste0(title, w),
             x = 'Number of users on the list',
             y = 'Count') +
        theme_light(base_size = 14) +
        geom_histogram(binwidth = 1);
        ggsave(paste0(graph_dir, filename, '_', w, graph_suffix),
               plot = g, width = 30, height = 20, units = "cm", dpi = "screen");
        graphs = append(graphs, g);
    }
    graphs;
}

In [142]:
%%R -i email_usage,email_usage_graph,email_usage_title,wikis,graph_folder,graph_suffix

graphs = make_histograms(data.table(email_usage), wikis, email_usage_title, email_usage_graph,
                graph_folder, graph_suffix);

In [136]:
%%R -i echo_usage,echo_usage_graph,echo_usage_title,wikis,graph_folder,graph_suffix

graphs = make_histograms(data.table(echo_usage), wikis, echo_usage_title, echo_usage_graph,
                graph_folder, graph_suffix);